In [2]:
import pandas as pd
import re

In [3]:
description_meta = pd.read_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_description/description_meta.parquet"
)
df_responses = pd.read_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_description/chat_responses.parquet"
)

# Check that the row counts match
if len(description_meta) != len(df_responses):
    print("Warning: The number of rows differ!")
else:
    print("Row counts match. Proceeding with validation...")


Row counts match. Proceeding with validation...


In [4]:
len(df_responses)

715

In [5]:
description_meta

,CANCER_TYPE_ACRONYM,ICD_10,ICD_O_3_HISTOLOGY,ICD_O_3_SITE,Normalized Sample Type,Case IDs,ICD_10_desc,ICD_O_3_SITE_desc,ICD_O_3_HISTOLOGY_desc,CANCER_TYPE_NAME
0,ACC,C74.0,8370/1,C74.0,"Blood Derived Normal, Primary Tumor","TCGA-OR-A5L9, TCGA-OR-A5J1, TCGA-OR-A5KU, TCGA...",Adrenal gland cortex,Adrenal gland cortex,None,Adrenocortical carcinoma
1,ACC,C74.0,8370/3,C74.0,"Blood Derived Normal, Primary Tumor","TCGA-OR-A5J8, TCGA-OR-A5KV, TCGA-OR-A5JP, TCGA...",Adrenal gland cortex,Adrenal gland cortex,Adrenal cortical carcinoma,Adrenocortical carcinoma
2,ACC,C74.0,8370/3,C74.0,"Primary Tumor, Solid Tissue Normal","TCGA-OR-A5KB, TCGA-PK-A5HC, TCGA-PK-A5HB, TCGA...",Adrenal gland cortex,Adrenal gland cortex,Adrenal cortical carcinoma,Adrenocortical carcinoma
3,BLCA,C67.0,8120/3,C67.0,"Blood Derived Normal, Primary Tumor","TCGA-DK-A2I6, TCGA-GV-A3QH, TCGA-FD-A62O, TCGA...","Trigone, bladder","Trigone, bladder","Transitional cell carcinoma, NOS",Bladder Urothelial Carcinoma
4,BLCA,C67.0,8120/3,C67.0,"Primary Tumor, Solid Tissue Normal","TCGA-BT-A20U, TCGA-BT-A20N, TCGA-BT-A20P, TCGA...","Trigone, bladder","Trigone, bladder","Transitional cell carcinoma, NOS",Bladder Urothelial Carcinoma
...,...,...,...,...,...,...,...,...,...,...
710,UCS,C54.1,8950/3,C54.1,"Primary Tumor, Solid Tissue Normal","TCGA-N6-A4VF, TCGA-N6-A4VE, TCGA-N6-A4VC, TCGA...",Endometrium,Endometrium,Mullerian mixed tumor,Uterine Carcinosarcoma
711,UCS,C54.1,8980/3,C54.1,"Blood Derived Normal, Primary Tumor",TCGA-N8-A4PI,Endometrium,Endometrium,"Carcinosarcoma, NOS",Uterine Carcinosarcoma
712,UCS,C54.2,8950/3,C54.2,"Blood Derived Normal, Primary Tumor",TCGA-N5-A59E,Myometrium,Myometrium,Mullerian mixed tumor,Uterine Carcinosarcoma
713,UCS,C54.3,8980/3,C54.3,"Blood Derived Normal, Primary Tumor",TCGA-N8-A4PM,Fundus uteri,Fundus uteri,"Carcinosarcoma, NOS",Uterine Carcinosarcoma


In [11]:
def extract_details(prompt):
    # Extract cancer type name, ICD-10 description, and tumor origin description
    pattern = r"Describe how (.*?) \(per ICD-10: (.*?)\) impacts survival, focusing on:\n- Tumor Origin: How (.*?) influences"
    match = re.search(pattern, prompt, re.DOTALL)
    if match:
        cancer_type_name = match.group(1).strip()
        icd10_desc = match.group(2).strip()
        icd_o3_site_desc = match.group(3).strip()
    else:
        cancer_type_name, icd10_desc, icd_o3_site_desc = None, None, None

    # Extract histology description
    pattern_histology = r"Histology: How (.*?) interacts with"
    match_histology = re.search(pattern_histology, prompt, re.DOTALL)
    icd_o3_histology_desc = match_histology.group(1).strip() if match_histology else None

    # Extract sampling bias (normalized sample type) details from the prompt
    pattern_sample = r"Sampling Bias: Limitations of (.*?) in genomic analysis"
    match_sample = re.search(pattern_sample, prompt, re.DOTALL)
    normalized_sample_type = match_sample.group(1).strip() if match_sample else None

    return {
        "CANCER_TYPE_NAME": cancer_type_name,
        "ICD_10_desc": icd10_desc,
        "ICD_O_3_SITE_desc": icd_o3_site_desc,
        "ICD_O_3_HISTOLOGY_desc": icd_o3_histology_desc,
        "Normalized_Sample_Type": normalized_sample_type
    }

In [14]:
df_responses

,prompt,response
0,\nresponse = client.chat.completions.create(\n...,### Tumor Origin\n\n**Influence on Survival:**...
1,\nresponse = client.chat.completions.create(\n...,### Tumor Origin: Adrenocortical Carcinoma\n\n...
2,\nresponse = client.chat.completions.create(\n...,**Adrenocortical Carcinoma (ACC) and Survival ...
3,\nresponse = client.chat.completions.create(\n...,To understand the survival impact of Bladder U...
4,\nresponse = client.chat.completions.create(\n...,### Tumor Origin: Trigone of the Bladder\n\n1....
...,...,...
710,\nresponse = client.chat.completions.create(\n...,1. **Tumor Origin:**\n - **Influences on Sur...
711,\nresponse = client.chat.completions.create(\n...,### Tumor Origin: Endometrium\n\n**Influence o...
712,\nresponse = client.chat.completions.create(\n...,### Uterine Carcinosarcoma and Survival Impact...
713,\nresponse = client.chat.completions.create(\n...,### Tumor Origin: Influence of Fundus Uteri on...


In [15]:
if "Normalized_Sample_Type" not in description_meta.columns:
    normalized_values = []
    for idx, row in df_responses.iterrows():
        details = extract_details(row["prompt"])
        normalized_values.append(details["Normalized_Sample_Type"])
    description_meta["Normalized_Sample_Type"] = normalized_values
    print("Added 'Normalized_Sample_Type' column to description_meta.")


Added 'Normalized_Sample_Type' column to description_meta.


In [16]:
n_check = min(3, len(df_responses))
for i in range(n_check):
    prompt = df_responses.loc[i, "prompt"]
    extracted = extract_details(prompt)
    original_row = description_meta.iloc[i]
    print(f"Row {i}:")
    for key, val in extracted.items():
        if key in description_meta.columns:
            original_val = str(original_row[key]).strip()
            if original_val != val:
                print(f"  MISMATCH in {key}: dataframe has '{original_val}' vs prompt extracted '{val}'")
            else:
                print(f"  {key} matches.")
        else:
            print(f"  Column '{key}' not found in description_meta!")
    print("-" * 40)

Row 0:
  CANCER_TYPE_NAME matches.
  ICD_10_desc matches.
  ICD_O_3_SITE_desc matches.
  MISMATCH in ICD_O_3_HISTOLOGY_desc: dataframe has 'None' vs prompt extracted 'UNKOWN'
  Normalized_Sample_Type matches.
----------------------------------------
Row 1:
  CANCER_TYPE_NAME matches.
  ICD_10_desc matches.
  ICD_O_3_SITE_desc matches.
  ICD_O_3_HISTOLOGY_desc matches.
  Normalized_Sample_Type matches.
----------------------------------------
Row 2:
  CANCER_TYPE_NAME matches.
  ICD_10_desc matches.
  ICD_O_3_SITE_desc matches.
  ICD_O_3_HISTOLOGY_desc matches.
  Normalized_Sample_Type matches.
----------------------------------------


In [ ]:
description_meta["generated_description"] = df_responses["response"].values

# Save the updated dataframe back to the parquet file
description_meta.to_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_description/description_meta_with_answers.parquet",
    index=False
)
description_meta.to_parquet(
    "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancertype_location_description/location_description/description_meta.parquet",
    index=False
)
print("Updated description_meta saved successfully.")

Updated description_meta saved successfully.


In [19]:
description_meta

,CANCER_TYPE_ACRONYM,ICD_10,ICD_O_3_HISTOLOGY,ICD_O_3_SITE,Normalized Sample Type,Case IDs,ICD_10_desc,ICD_O_3_SITE_desc,ICD_O_3_HISTOLOGY_desc,CANCER_TYPE_NAME,Normalized_Sample_Type,generated_description
0,ACC,C74.0,8370/1,C74.0,"Blood Derived Normal, Primary Tumor","TCGA-OR-A5L9, TCGA-OR-A5J1, TCGA-OR-A5KU, TCGA...",Adrenal gland cortex,Adrenal gland cortex,None,Adrenocortical carcinoma,"Blood Derived Normal, Primary Tumor",### Tumor Origin\n\n**Influence on Survival:**...
1,ACC,C74.0,8370/3,C74.0,"Blood Derived Normal, Primary Tumor","TCGA-OR-A5J8, TCGA-OR-A5KV, TCGA-OR-A5JP, TCGA...",Adrenal gland cortex,Adrenal gland cortex,Adrenal cortical carcinoma,Adrenocortical carcinoma,"Blood Derived Normal, Primary Tumor",### Tumor Origin: Adrenocortical Carcinoma\n\n...
2,ACC,C74.0,8370/3,C74.0,"Primary Tumor, Solid Tissue Normal","TCGA-OR-A5KB, TCGA-PK-A5HC, TCGA-PK-A5HB, TCGA...",Adrenal gland cortex,Adrenal gland cortex,Adrenal cortical carcinoma,Adrenocortical carcinoma,"Primary Tumor, Solid Tissue Normal",**Adrenocortical Carcinoma (ACC) and Survival ...
3,BLCA,C67.0,8120/3,C67.0,"Blood Derived Normal, Primary Tumor","TCGA-DK-A2I6, TCGA-GV-A3QH, TCGA-FD-A62O, TCGA...","Trigone, bladder","Trigone, bladder","Transitional cell carcinoma, NOS",Bladder Urothelial Carcinoma,"Blood Derived Normal, Primary Tumor",To understand the survival impact of Bladder U...
4,BLCA,C67.0,8120/3,C67.0,"Primary Tumor, Solid Tissue Normal","TCGA-BT-A20U, TCGA-BT-A20N, TCGA-BT-A20P, TCGA...","Trigone, bladder","Trigone, bladder","Transitional cell carcinoma, NOS",Bladder Urothelial Carcinoma,"Primary Tumor, Solid Tissue Normal",### Tumor Origin: Trigone of the Bladder\n\n1....
...,...,...,...,...,...,...,...,...,...,...,...,...
710,UCS,C54.1,8950/3,C54.1,"Primary Tumor, Solid Tissue Normal","TCGA-N6-A4VF, TCGA-N6-A4VE, TCGA-N6-A4VC, TCGA...",Endometrium,Endometrium,Mullerian mixed tumor,Uterine Carcinosarcoma,"Primary Tumor, Solid Tissue Normal",1. **Tumor Origin:**\n - **Influences on Sur...
711,UCS,C54.1,8980/3,C54.1,"Blood Derived Normal, Primary Tumor",TCGA-N8-A4PI,Endometrium,Endometrium,"Carcinosarcoma, NOS",Uterine Carcinosarcoma,"Blood Derived Normal, Primary Tumor",### Tumor Origin: Endometrium\n\n**Influence o...
712,UCS,C54.2,8950/3,C54.2,"Blood Derived Normal, Primary Tumor",TCGA-N5-A59E,Myometrium,Myometrium,Mullerian mixed tumor,Uterine Carcinosarcoma,"Blood Derived Normal, Primary Tumor",### Uterine Carcinosarcoma and Survival Impact...
713,UCS,C54.3,8980/3,C54.3,"Blood Derived Normal, Primary Tumor",TCGA-N8-A4PM,Fundus uteri,Fundus uteri,"Carcinosarcoma, NOS",Uterine Carcinosarcoma,"Blood Derived Normal, Primary Tumor",### Tumor Origin: Influence of Fundus Uteri on...
